Task#1: Get Info Box(Store in python dictionary)

Import necessary libraries

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

Load the webpage

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

# convert to a beautiful soup object

soup = bs(r.content)

contents = soup.prettify()

print(contents)

In [ ]:
info_box = soup.find(class_ = "infobox vevent")

info_row = info_box.find_all("tr")

for row in info_row:
    print(row.prettify())


In [ ]:
def get_content_value(row_data):
    if row_data.find("li"):
        return[li.get_text(" ", strip = True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")

movie_info = {}
for index, row in enumerate(info_row):
    if index == 0:
        movie_info['title'] = row.find('th').get_text(" ", strip = True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip = True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value

    
movie_info
    

Task#2: Get Info Box for all Movies

Scap the whole list of movies

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

# convert to a beautiful soup object

soup = bs(r.content)

contents = soup.prettify()

print(contents)

In [ ]:
# Method1

movies = soup.find_all(class_ = "wikitable sortable")
movies

In [ ]:
# Method2

movies = soup.select(".wikitable.sortable i")
movies[1:10]

In [ ]:
movies[0]

In [ ]:
movies[0].a['href']

In [ ]:
movies[0].a['title']

In [ ]:
def get_content_value(row_data):
    if row_data.find("li"):
        return[li.get_text(" ", strip = True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    
    else:
        return row_data.get_text(" ", strip = True).replace("\xa0", " ")
    
def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()

def get_info_box(url):
    
    r = requests.get(url)
    
    soup = bs(r.content)
    contents = soup.prettify()
    info_box = soup.find(class_ = "infobox vevent")
    info_row = info_box.find_all("tr")
    
    clean_tags(soup)
    
    
    movie_info = {}
    for index, row in enumerate(info_row):
        if index == 0:
            movie_info['title'] = row.find('th').get_text(" ", strip = True)
       
        else:
            header = row.find("th")
            if header:
                content_key = row.find("th").get_text(" ", strip = True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value

    
    return movie_info

In [ ]:
get_info_box("https://en.wikipedia.org/wiki/Secrets_of_Life")

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")
movies[0]

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")
base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info_box(full_path))
    except Exception as e:
        print(movie.get_text())
        print(e)


In [ ]:
movie_info_list

In [ ]:
len(movie_info_list)

Save/Reload movie data

In [ ]:
# save data
import json

def save_data(title, data):
    with open(title, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii = False, indent  = 2)

In [ ]:
# load data

import json

def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

In [ ]:
save_data('desney_data_cleaned.json', movie_info_list)

In [ ]:
movie_info_list = load_data('desney_data_cleaned.json')

In [ ]:
# load data

import json

def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

Task#3: Cleaning of data

In [ ]:
#### Subtasks

#1: ~strikeCleaning up references
#2: converting running time into integers
#3: converting date into datetime object
#4: split up the long strings
#5: Converting budget and box office into numbers

In [ ]:
#3: converting date into datetime object

In [ ]:
[movie.get("Running time", "N/A") for movie in movie_info_list]

In [ ]:
def minutes_to_integer(running_time):
    if running_time == 'N/A':
        return None
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else:
        return int(running_time.split(" ")[0])

for movie in movie_info_list:
    movie["Running time (int)"] = minutes_to_integer(movie.get('Running time', 'N/A'))

In [ ]:
print([movie.get("Running time (int)", "N/A") for movie in movie_info_list])

In [ ]:
#4: Converting budget and box office into numbers

In [ ]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"
standard = fr"\${number}(-|\sto\s)?({number})?\s({amounts})"

def word_to_value(word):
    value_dict = {"thousand": 1000, "million": 1000000, "billion": 1000000000}
    return value_dict.get(word.lower(), 1)

def parse_word_syntax(string):
    stripped_string = string.replace(",", "")
    value = float(re.search(number, stripped_string).group())
    modifier = word_to_value(re.search(amounts, string, flags=re.I).group())
    return value*modifier

def parse_value_syntax(string):
    stripped_string = string.replace(",", "")
    return float(re.search(number, stripped_string).group())

def money_conversion(money):
    if type(money) == list:
        money = money[0]

    word_syntax = re.search(standard, money, flags=re.I)
    value_syntax = re.search(fr"\${number}", money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())
    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    else:
        return None

print(money_conversion("$790 Million"))    

In [ ]:
for movie in movie_info_list:
    movie["Budget (float)"] = money_conversion(movie.get('Budget', 'N/A'))
    movie["Box office (float)"] = money_conversion(movie.get('Box office', 'N/A'))

In [ ]:
movie_info_list[-40]

In [ ]:
[movie.get("Budget", "N/A") for movie in movie_info_list]

#5: converting data to datatime objects

In [ ]:
movie_info_list[70]

In [ ]:
[movie.get("Release date", "N/A") for movie in movie_info_list]

In [ ]:
from datetime import datetime

dates = [movie.get('Release date', 'N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split('(')[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
    
    if date == 'N/A':
        return None
        
    date_str = clean_date(date)
    
    fmts  = ["%B %d, %Y", "%d %B %Y"]
    
    for fmt in fmts:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            pass
    return None

In [ ]:
for movie in movie_info_list:
    movie['Release date (datetime)'] = date_conversion(movie.get('Release date', 'N/A'))

In [ ]:
movie_info_list[300]

In [ ]:
import pickle

def save_data_pickle(name, data):
    with open(name, 'wb') as f:
        pickle.dump(data, f)

In [ ]:
import pickle

def load_data_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [ ]:
save_data_pickle("desney_data_cleaned_more.pickle", movie_info_list)

In [ ]:
a = load_data_pickle("desney_data_cleaned_more.pickle")

In [ ]:
a[5]

Task#4: Attach IMDB/Rotten tomatoes scores

In [ ]:
movie_info_list = load_data_pickle("desney_data_cleaned_more.pickle")

In [ ]:
def get_movie_data(title):
    baseurl=' http://www.omdbapi.com/?i=tt3896198&apikey=9d8220a' #Enter your API key here
    params_diction = {}
    params_diction['t']=title
    params_diction['r']='json'
    response = requests.get(baseurl, params=params_diction)
    #print(response.url)
    return response.json()
# get_movie_data('the nutcracker and the four realms')

def rotten_tomato_score(omdb_info):
    ratings = omdb_info.get('Ratings', [])
    for rating in ratings:
        if rating['Source'] == 'Rotten Tomatoes':
            return rating['Value']
    return None

get_movie_data('the nutcracker and the four realms')

In [ ]:
for movie in movie_info_list:
    title = movie['title']
    omdb_info = get_movie_data(title)
    movie['imdb'] = omdb_info.get('imdbRating', None)
    movie['genre'] = omdb_info.get('Genre', None)
    movie['metascore'] = omdb_info.get('Metascore', None)
    movie['rotten_tomatoes'] = rotten_tomato_score(omdb_info)
    
    

In [ ]:
movie_info_list[-50]

In [ ]:
save_data_pickle("desney_data_cleaned_final.pickle", movie_info_list)

task# 5: Save data in json and csv file

In [ ]:
movie_info_list[-50]

In [ ]:
movie_info_list_copy = [movie.copy() for movie in movie_info_list]

In [ ]:
for movie in movie_info_list_copy:
    current_date = movie['Release date (datetime)']
    if current_date:
        movie['Release date (datetime)'] = current_date.strftime("%B %d, %Y")
    else:
        movie['Release date (datetime)'] = None

In [ ]:
movie_info_list_copy[20]

In [ ]:
save_data("disney_data_final.json", movie_info_list_copy)

####Converting Data to CSV

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)

df = pd.DataFrame(movie_info_list)

In [ ]:
df.head()

In [ ]:
df.to_csv("disney_movie_data_final.csv")

In [ ]:
df.info()

In [ ]:
running_times = df.sort_values(['Running time (int)'], ascending = False)
running_times.head(30)